# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

## Data Exploration

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as sp

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [4]:
data.shape

(4870, 65)

In [5]:
df=data.groupby(['race','call']).size().reset_index(name='counts')
df

,race,call,counts
0,b,0.0,2278
1,b,1.0,157
2,w,0.0,2200
3,w,1.0,235


## Solution

 #### 1. What test is appropriate for this problem? Does CLT apply?
 
 Two sample T test is appropriate for this problem to compare the mean call back rate of the two different groups. The two different race groups can be treated as independent of each other. Central limit theorem applies to this as the sample size is quite larger than 30. Hence, the mean of the sampling distribution of the sample proportion is going to be close to the population mean.

 #### 2. What are the null and alternate hypotheses?
     - Null Hypothesis: There is no difference between of proportion of white candidates and proportion of black candidates receiving call back 
     - Alternative Hypothesis: There is a difference between of proportion of white candidates and proportion of black candidates receiving call back 

#### 3. Compute margin of error, confidence interval, and p-value.

For a Bernoulli distribution, the proportion of success is same as mean. Call back can only have two values, 1 for success and 0 for fail which makes it a Bernoulli trial

We are going to check if the difference between the means/proportions of white and black receiving call back is significant or not.

##### a. Proportion of white candidates that were called

In [6]:
p_w_call=df[(df['call']==1.0) & (df['race']=='w')]['counts']/df[(df['race']=='b')].sum()['counts']
p_w_call

3    0.096509
Name: counts, dtype: float64

##### b.Variance of proportion of white candidates that were called

In [7]:
var_w=(sp.bernoulli.var(p_w_call, loc=0))/(df[(df['race']=='w')].sum()['counts'])
var_w

array([  3.58091198e-05])

##### c. Proportion of black candidates that were called

In [8]:
p_b_call=df[(df['call']==1.0) & (df['race']=='b')]['counts']/df[(df['race']=='b')].sum()['counts']
p_b_call

1    0.064476
Name: counts, dtype: float64

##### d. Variance of proportion of black candidates that were called

In [9]:
var_b=(sp.bernoulli.var(p_b_call, loc=0))/(df[(df['race']=='b')].sum()['counts'])
var_b

array([  2.47717379e-05])

##### e. Difference in proportion of white candidates and proportion of black candidates that were called=
##### Proportion of white candidates that were called - Proportion of black candidates that were called


In [10]:
p_diff= np.subtract(p_w_call,p_b_call)
print(p_diff)

3    0.032033
Name: counts, dtype: float64


##### Variance in difference between proportion of white candidates and proportion of blacks candidates that were called=
##### Variance in proportion of white candidates that were called + Variance in proportion of black candidates that were called
##### In the below code, Standard deviation and Confidence interval is also calculated
##### 1.96 is the Z score of the 97.5 percentile value

In [11]:
#Add both variances
var_diff=np.add(var_w,var_b)
#Calculate standard deviation as the square root of variance
sd_diff=np.sqrt(var_diff)

#distance between the mean and the 97.5 percentile value
d=np.multiply(1.96,sd_diff)

#97.5 confidence interval point
print(np.subtract(p_diff,d))
#2.5 confidence interval point
print(np.add(p_diff,d))

3    0.016777
Name: counts, dtype: float64
3    0.047288
Name: counts, dtype: float64


#### There is a 95% chance that the difference between the proportion of white candidates getting a call and proportion of black candidates getting a call is between 0.016777 and 0.047288

 ##### Revisiting null hypothesis
 - ##### Null Hypothesis: There is no difference between of proportion of white candidates and proportion of black candidates receiving call back 
 - ##### Alternative Hypothesis: There is a difference between of proportion of white candidates and proportion of black candidates receiving call back 
 
 ##### If the probability of getting a difference of the two proportions as 'Zero' is less than 5% (alpha), we will reject the null hypothesis
 
 ##### Assuming that there is NO difference in the proportions of white and black candidates receiving call backs, what is the probability of getting a difference of mean as extreme or more than what we found in the sample i.e 0.032033? For this we will have to calculate Z Score for the value of 0.032033 and if the P-value is less than 0.05 we can reject the null hypothesis.

##### Z= Difference sample proportions- 0 / standard deviation of difference of sample proportions given there is no difference between sample proportions

If hypothesis is true, 

##### p_w_call=p_b_call=p
##### f. Calculate p

In [12]:
#P is the proportion of call backs for the entire data
p=df[(df['call']==1.0)].sum()['counts']/df.sum()['counts']
p

0.08049281314168377

##### g. Calculate variance now considering p_w_call=p_b_call, after further simplication following formula can be applied

In [13]:
var_z=np.divide((float(2.0)*float(p)*float(np.subtract(1,p))),(df[(df['race']=='b')].sum()['counts']))
std_z=np.sqrt(var_z)
z=p_diff/std_z
z                                  

3    4.108412
Name: counts, dtype: float64

### Z Score of 4 indicates that the probability of having NO difference in proportion of white and black candidates receiving call back highly unlikely

Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

##### This was an experimental study with emphasis on Race specific names and to find out the difference in perception of the race. There might be other correlations in the data to the callback success but that would still have to be performed in the context of race.

## Story


The goal of the statistical testing is to find out if white candidates receive more call backs as compared to black candidates for the same qualifications. 

The data has 4870 samples out of which one of the resumes have a white sounding name and the other half has the same resumes with black sounding names.

To test the difference between the call backs received by black or white candidates, Two sample T test has been used as both the groups of resumes can be considered independent.

In the first part of the test, difference in proportions in the given sample and the standard deviation and 95% confidence interval is calculated. The difference in proportions is 0.032033 and the 95% confidence interval is between 0.016777 and 0.047288 which indicates that there is 95% chance that white sounding resumes receive more call backs than the black sounding ones as the lower limit of the confidence interval is positive.

In the next part, the null hypothesis which is "There is NO difference in callback rate for White candidates and Black candidates" is tested . I calculated the Z score of the mean difference that I got in step # 3 of 0.032033, to determine how likely it is to get a value as extreme or more as 0.032033 assuming the null hypothesis is true.

The Z score calculated is greater than 4 which clearly indicates that the null hypothesis can be rejected. We can conclude that there is indeed a difference in call backs among black and white candidates.

### Consulted References
Bernoulli's distribution
https://www.khanacademy.org/math/statistics-probability/sampling-distributions-library/sample-proportions/v/bernoulli-distribution-mean-and-variance-formulas

Comparing Population Proportion
https://www.khanacademy.org/math/statistics-probability/significance-tests-confidence-intervals-two-samples/v/comparing-population-proportions-1

Hypothesis test for compating population proportions
https://www.khanacademy.org/math/statistics-probability/significance-tests-confidence-intervals-two-samples/comparing-two-proportions/v/hypothesis-test-comparing-population-proportions

Hypothesis Test
https://www.khanacademy.org/math/statistics-probability/significance-tests-one-sample/tests-about-population-mean/v/hypothesis-testing-and-p-values

Bernoulli functions Python
https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.bernoulli.html


Two sample vs paired t tests
http://www.statisticshowto.com/probability-and-statistics/t-test/

Datacamp statistical thinking part 2: A two-sample bootstrap hypothesis test for difference of means

https://www.cliffsnotes.com/study-guides/statistics/univariate-inferential-tests/two-sample-t-test-for-comparing-two-means